In [3]:
# %matplotlib widget 

import torch
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn

import pandas as pd

from utils import PrCol
from bnn import BNN, BNNDataset
import copy
import os

import matplotlib.lines as mlines

Color = PrCol()

TRAIN_BNN_LF_FLAG = False
TRAIN_BNN_MF_FLAG = False
TRAIN_BNN_HF_FLAG = False
TRAIN_TL_FLAG = False
TRAIN_TL_FT_FLAG = False

SAVE_MODEL_FLAG = False

DEVICE = torch.device("cpu")

dataset_location = "Dataset/bscw_dataset.csv"
SEP = ";"
input_labels = ["mach","aoa"]
output_labels = ["cl","cm"]
fidelity_column_name = "fidelity"
fidelities = ["low","mid","cfd"]

study_name = "BSCW_dataset_1"
model_name_prefix = "BSCW"


In [4]:
study_path = f"AIModels/{study_name}"

try: os.makedirs(study_path)
except FileExistsError: pass

In [5]:
dataset_all = pd.read_csv(dataset_location,sep=SEP)
display(dataset_all)

,mach,aoa,cl,cm,fidelity
0,0.700,0.000000,0.222070,-1.443585e-05,low
1,0.700,0.166667,0.248369,4.219838e-06,low
2,0.700,0.333333,0.275482,-8.271854e-07,low
3,0.700,0.500000,0.297021,9.484167e-08,low
4,0.700,0.666667,0.318354,9.272213e-10,low
...,...,...,...,...,...
724,0.737,0.819000,0.251295,-7.164018e-02,cfd
725,0.766,2.300000,0.386129,-6.008131e-02,cfd
726,0.758,3.109000,0.454622,-5.301218e-02,cfd
727,0.771,2.570000,0.412188,-5.756897e-02,cfd


In [7]:
dataset_lf_denorm = dataset_all[dataset_all[fidelity_column_name] == "low"]
dataset_mf_denorm = dataset_all[dataset_all[fidelity_column_name] == "mid"]
dataset_hf_denorm = dataset_all[dataset_all[fidelity_column_name] == "cfd"]

In [8]:
from scalers import MinMaxScaler as MMS

scaler = MMS(dataset_all.drop(columns=[fidelity_column_name], inplace=False),interval=(1,2))
normalized_datasets = []
for fid in fidelities:
    dataset_fidelity = dataset_all[dataset_all[fidelity_column_name] == fid]
    dataset_fidelity_norm = scaler.scaleDataframe(dataset_fidelity.drop(columns=[fidelity_column_name], inplace=False))
    normalized_datasets.append(dataset_fidelity_norm)
scaler.save(f"{model_name_prefix}norm.pkl", path=study_path)

In [9]:
dataset_lf = BNNDataset(normalized_datasets[0],input_labels,output_labels,device=DEVICE)
dataset_mf = BNNDataset(normalized_datasets[1],input_labels,output_labels,device=DEVICE)
dataset_hf = BNNDataset(normalized_datasets[2],input_labels,output_labels,device=DEVICE)
dataset_all = BNNDataset(pd.concat([normalized_datasets[0], normalized_datasets[1], normalized_datasets[2]], axis=0),input_labels,output_labels,device=DEVICE)

train_lf, valid_lf = dataset_lf.train_val_split(seed=42)
train_mf, valid_mf = dataset_mf.train_val_split(train_size=0.4,seed=42)
train_hf, valid_hf, test_hf = dataset_hf.train_val_test_split(train_size=0.06,val_size=0.04,seed=42)
max_rows = dataset_hf.data.nlargest(2, ['mach', 'aoa'])
train_hf = BNNDataset(pd.concat([train_hf.data,max_rows]),input_labels,output_labels,device=DEVICE)

print("Low:", len(train_lf), len(valid_lf))
print("Mid:", len(train_mf), len(valid_mf))
print("Hig:", len(train_hf), len(valid_hf), len(test_hf))

Low: 437 188
Mid: 19 29
Hig: 5 2 51


### Train the BNN on Low Fidelity Data

In [12]:
model_path = f"{study_path}/{model_name_prefix}_bnn_lf.pt"

bnn_lf_model = BNN(
    in_dim=len(input_labels),
    out_dim=len(output_labels),
    mu = 0,
    std = 0.0126,
    units = [176,176,144,176,176],
    denseOut = False,
    dropout = False,
    device=DEVICE,
    activation=nn.LeakyReLU(),
    model_name="BNN LF"
)
if TRAIN_BNN_LF_FLAG:
    bnn_lf_model.train_model(
        train_lf,
        valid_lf,
        patience=150,
        n_epochs=5000,
        batch_size = 32
    )
    if SAVE_MODEL_FLAG: bnn_lf_model.save(model_path)
else: bnn_lf_model.load(model_path)

[BNN LF] >> Model Loaded on cpu


### Train the BNN on Mid Fidelity Data

In [13]:
model_path = f"{study_path}/{model_name_prefix}_bnn_mf.pt"

bnn_mf_model = BNN(
    in_dim=len(input_labels),
    out_dim=len(output_labels),
    mu = 0,
    std = 0.0526,
    units = [160,80],
    denseOut = False,
    dropout = False,
    device=DEVICE,
    activation = nn.LeakyReLU(),
    model_name = "BNN MF"
)
if TRAIN_BNN_MF_FLAG:
    bnn_mf_model.train_model(
        train_mf,
        valid_mf,
        patience=500,
        n_epochs=10000,
        batch_size = 32
    )
    if SAVE_MODEL_FLAG: bnn_mf_model.save(model_path)
else: bnn_mf_model.load(model_path)

[BNN MF] >> Model Loaded on cpu


### Train the BNN on High Fidelity Data

In [14]:
model_path = f"{study_path}/{model_name_prefix}_bnn_hf.pt"

bnn_hf_model = BNN(
    in_dim=len(input_labels),
    out_dim=len(output_labels),
    mu = 0,
    std = 0.0596,
    units = [128,160,176],
    denseOut = False,
    dropout = False,
    device=DEVICE,
    activation = nn.LeakyReLU(),
    model_name = "BNN HF"
)
if TRAIN_BNN_HF_FLAG:
    bnn_hf_model.train_model(
        train_hf,
        valid_hf,
        patience=1000,
        n_epochs=100000,
        batch_size = 32
    )
    if SAVE_MODEL_FLAG: bnn_hf_model.save(model_path)
else: bnn_hf_model.load(model_path)

[BNN HF] >> Model Loaded on cpu


### Do the Transfer Learning

In [15]:
bnn_df_model = copy.deepcopy(bnn_lf_model)
bnn_df_model.model_name = "BNN TL"
model_path_1 = f"{study_path}/{model_name_prefix}_bnn_tl_2.pt"
model_path_2 = f"{study_path}/{model_name_prefix}_bnn_tl_finetune_2.pt"

if TRAIN_TL_FLAG:
    #FREEZE ALL THE PARAMS IN THE MODEL
    bnn_df_model.setModelGradients(False)
    all_layers = bnn_df_model.getAllLayersName()
    layers_to_unfreeze = all_layers[-4:]
    print("Unfreezing",layers_to_unfreeze)
    bnn_df_model.setModelGradients(True,layers=layers_to_unfreeze)
    #Train with MF
    bnn_df_model.train_model(
        train_mf,
        valid_mf,
        n_epochs=8000,
        lr=0.001,
        restoreBestModel=True,
        patience=500,
        batch_size = 32,
        earlyStopping=True
    )
    if SAVE_MODEL_FLAG: bnn_df_model.save(model_path_1)
else: 
    bnn_df_model.load(model_path_1)
    bnn_df_model_fine_tuned = copy.deepcopy(bnn_df_model) 
    bnn_df_model_fine_tuned.model_name = "BNN TL FT"

if TRAIN_TL_FT_FLAG:
    bnn_df_model_fine_tuned = copy.deepcopy(bnn_df_model) 
    bnn_df_model_fine_tuned.model_name = "BNN TL FT"
    bnn_df_model.setModelGradients(False)
    layers_to_unfreeze = all_layers[-2:]
    bnn_df_model.setModelGradients(True,layers=layers_to_unfreeze)
    bnn_df_model_fine_tuned.train_model(
        train_hf+valid_hf,
        train_hf+valid_hf,
        n_epochs=8000,
        lr=0.001,
        restoreBestModel=True,
        patience=500,
        batch_size = 1,
        earlyStopping=True
    )
    if SAVE_MODEL_FLAG: bnn_df_model_fine_tuned.save(model_path_2)
else: 
    bnn_df_model_fine_tuned.load(model_path_2)

[BNN TL] >> Model Loaded on cpu
[BNN TL FT] >> Model Loaded on cpu


In [ ]:
pred_mean, pred_std = bnn_df_model_fine_tuned.predict(test_hf.data[["mach","aoa"]].values,scaler=scaler,output_labels=output_labels,attempt=100)